In [3]:
import concurrent.futures
from test1 import TestFunct
if __name__ == "__main__":
            with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
                results = [executor.submit(TestFunct, "chicken butt") for _ in range(3)] 
                for result in concurrent.futures.as_completed(results):
                    print(result.result())
                    # flights.append(result.result())
                    # flightForCurrent.append(result.result())   

chicken butt
chicken butt
chicken butt
